In [1]:
import os
from PIL import Image
import logging

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from transformers import CLIPModel, CLIPProcessor
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

comp_prefix = "/home/hutu/workspace/MMRetrieval/data"
train_path = comp_prefix + "/train.csv"
test_path = comp_prefix + "/test.csv"

CV = True if len(pd.read_csv(test_path)) < 10 else False

img_dir = "/home/hutu/workspace/MMRetrieval/data/train_images" if CV else "/home/hutu/workspace/MMRetrieval/data/test_images"
split = "valid"

use_CLIP = False
use_BGE = True

model2path = {
    "clip": "/home/hutu/workspace/MMRetrieval/modeloutput/1211/infoNCE/checkpoint-1080",
    "bge": "/home/hutu/workspace/SentenceEmbedding/modeloutput/1214/checkpoint-2142",
}


## Utils function and Class

In [3]:
def visualize(df, index, col):
    """可视化图片和预测结果，预测结果为posting_id的list"""
    row = df.iloc[index]
    preds = row[col]
    img_dir: str = "/home/hutu/workspace/MMRetrieval/data/train_images"
    images = [df[df.posting_id == pred].image.values[0] for pred in preds]

    target_title = row.title
    target_img = row.image

    titles = [df[df.posting_id == pred].title.values[0] for pred in preds]
    images = [Image.open(os.path.join(img_dir, img)) for img in images]

    rows = 5
    cols = 5
    fig, ax = plt.subplots(rows, cols, figsize=(20, 20))

    plt.axis("off")

    for i in range(cols):
        ax[0, i].axis("off")

    for i in range(rows):
        ax[i, 0].axis("off")

    ax[0, 0].imshow(Image.open(os.path.join(img_dir, target_img)))
    ax[0, 0].set_title(
        "\n".join([target_title[i : i + 10] for i in range(0, len(target_title), 10)]),
        fontsize=12,
    )

    for i in range(1, rows):
        for j in range(1, cols):
            idx = (i - 1) * cols + (j - 1)
            ax[i, j].axis("off")
            if idx < len(images):
                ax[i, j].imshow(images[idx])
                ax[i, j].set_title(
                    "\n".join(
                        [
                            titles[idx][k : k + 10]
                            for k in range(0, len(titles[idx]), 10)
                        ]
                    ),
                    fontsize=12,
                )


def compute_f1(col):
    def f1(row):
        n = len(np.intersect1d(row["label"], row[col]))
        return 2 * n / (len(row["label"]) + len(row[col]))

    return f1


def compute_recall(col):
    def recall(row):
        n = len(np.intersect1d(row["label"], row[col]))
        return n / len(row["label"])

    return recall


def compute_precision(col):
    def precision(row):
        n = len(np.intersect1d(row["label"], row[col]))
        return n / len(row[col])

    return precision


def compute_precision_K(col, K):
    def precision(row):
        n = len(np.intersect1d(row["label"], row[col][:K]))
        return n / K

    return precision


def compute_AP(col, N):
    """compute average precision"""

    def AP(row):
        n = len(np.intersect1d(row["label"], row[col]))
        max_n = min(len(row[col]), N)
        if n == 0:
            return 0
        return (
            sum(
                [
                    compute_precision_K(col, i)(row)
                    for i in range(1, max_n + 1)
                    if row[col][i - 1] in row["label"]
                ]
            )
            / max_n
        )

    return AP

In [4]:
class ShopeeDataset(Dataset):
    def __init__(self, df, img_dir, split: str = "train"):
        self.df = df
        self.img_dir = img_dir
        self.transform = transforms.Compose(
            [
                transforms.Resize(256),
                transforms.CenterCrop(224),
                # convert to RGB
                transforms.Lambda(lambda img: img.convert("RGB")),
                transforms.ToTensor(),
            ]
        )
        self.split = split
        self.len = len(self.df)
        # self.imgs = self._read_all_images()

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        if self.split == "train":
            query_text = row["query"]
            pos_text = row["pos_txt"][0]
            neg_text = row["neg_txt"][0]

            query_img_path = os.path.join(self.img_dir, row["image"])
            pos_img_path = os.path.join(self.img_dir, row["pos_img"][0])
            neg_img_path = os.path.join(self.img_dir, row["neg_img"][0])

            query_img = self._get_image(query_img_path)
            pos_img = self._get_image(pos_img_path)
            neg_img = self._get_image(neg_img_path)

            return {
                "query": {
                    "text": query_text,
                    "image": query_img,
                },
                "pos": {
                    "text": pos_text,
                    "image": pos_img,
                },
                "neg": {
                    "text": neg_text,
                    "image": neg_img,
                },
            }

        elif self.split == "valid":
            title = row["title"]
            image_path = os.path.join(self.img_dir, row["image"])

            img = self._get_image(image_path)

            return {
                "title": title,
                "image": img,
            }

    def _get_image(self, path):
        img = Image.open(path)
        img = self.transform(img)
        return img

In [5]:
def retrieval(embs, threshold, df, chunk_size=4096):
    embs_pt = torch.tensor(embs).cuda()

    num_chunks = (embs_pt.shape[0] + chunk_size - 1) // chunk_size
    posting_id = df.posting_id.to_list()
    topk_posting_id = []

    print(f"Chunk Size: {chunk_size}, {num_chunks} chunks")


    for i in tqdm(range(num_chunks)):
        start = i * chunk_size
        end = min((i + 1) * chunk_size, embs_pt.shape[0])
        sim = embs_pt[start:end] @ embs_pt.T
        mask = sim > threshold

        indices = [
            torch.nonzero(mask[j]).squeeze().cpu().numpy() for j in range(mask.shape[0])
        ]
        indices = [np.unique(i) for i in indices]
        sorted_indices = [
            indices[j][np.argsort(-sim[j, indices[j]].cpu().numpy())]
            for j in range(len(indices))
        ]

        topk_posting_id.extend(
            [[posting_id[j] for j in row] for row in sorted_indices]
        )

    # clean up
    del embs_pt
    torch.cuda.empty_cache()
    
    return topk_posting_id

def neighbor_blend(embs, threshold, chunk_size=4096):
    embs_pt = torch.tensor(embs).cuda()
    num_chunks = (embs_pt.shape[0] + chunk_size - 1) // chunk_size
    res_embs = []
    
    print(f"Chunk Size: {chunk_size}, {num_chunks} chunks")

    for i in tqdm(range(num_chunks)):
        start = i * chunk_size
        end = min((i + 1) * chunk_size, embs_pt.shape[0])
        sim = embs_pt[start:end] @ embs_pt.T
        mask = sim > threshold

        indices = [
            torch.nonzero(mask[j]).squeeze().cpu().numpy() for j in range(mask.shape[0])
        ]
        indices = [np.unique(i) for i in indices]
        
        # blend embs according to the similarity
        for j in range(len(indices)):
            weights = sim[j, indices[j]].softmax(dim=0)
            res_embs.append((weights.unsqueeze(1) * embs_pt[indices[j]]).sum(dim=0).cpu().numpy())
        
    # clean up
    del embs_pt
    torch.cuda.empty_cache()

    res_embs = np.stack(res_embs)
    res_embs = res_embs / np.linalg.norm(res_embs, axis=1, keepdims=True)


    return res_embs

## Load csv to df

In [6]:
df = pd.read_csv(train_path) if CV else pd.read_csv(test_path)

df.head(2)

,posting_id,image,image_phash,title,label_group
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045


In [7]:
if CV:
    tmp = df.groupby("label_group").posting_id.agg("unique").to_dict()
    df["label"] = df.label_group.map(tmp)

tmp = df.groupby("image_phash").posting_id.agg("unique").to_dict()
df["phash"] = df.image_phash.map(tmp)

df.head(2)

,posting_id,image,image_phash,title,label_group,label,phash
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,"[train_129225211, train_2278313361]",[train_129225211]
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,"[train_3386243561, train_3423213080]",[train_3386243561]


In [8]:
if CV:
    df["f1"] = df.apply(compute_f1("phash"), axis=1)
    df["recall"] = df.apply(compute_recall("phash"), axis=1)
    df["precision"] = df.apply(compute_precision("phash"), axis=1)
    df["AP"] = df.apply(compute_AP("phash", 50), axis=1)
    
    print(df.f1.mean(), df.recall.mean(), df.precision.mean(), df.AP.mean())

0.5530933399168149 0.4221722749534632 0.9940677878488098 0.9923105721349715


## CLIP Model linear probe

In [9]:
class CLIPForEmbedding(CLIPModel):
    def forward(
        self,
        inputs,
    ):
        text_inputs = inputs["text"]
        image_inputs = inputs["image"]

        text_embs = self.get_text_features(**text_inputs)
        image_embs = self.get_image_features(**image_inputs)

        return {"text": text_embs, "image": image_embs}

class CLIPForFusion(nn.Module):
    def __init__(self, clip_model=None, processor=None):
        super().__init__()
        self.clip_model = clip_model
        self.processor = processor

        self.txt_hidden_dim = 768
        self.img_hidden_dim = 768
        self.fusion_dim = 512

        self.text_fc = nn.Linear(self.txt_hidden_dim, self.fusion_dim)
        self.image_fc = nn.Linear(self.img_hidden_dim, self.fusion_dim)
        self.act_fn = nn.ReLU()

        self.fusion_fc = nn.Linear(self.fusion_dim * 2, self.fusion_dim)

    @classmethod
    def from_pretrained(cls, model_name_or_path):
        clip_model = CLIPForEmbedding.from_pretrained(model_name_or_path)
        processor = CLIPProcessor.from_pretrained(model_name_or_path)
        model = cls(clip_model, processor)
        model_path = os.path.join(model_name_or_path, "model.pt")

        if os.path.exists(model_path):
            logging.info(f"Load model.pt from {model_path}")
            state_dict = torch.load(model_path, weights_only=True)
            # only load the fc layers
            missing_keys, unexpected_keys = model.load_state_dict(
                state_dict, strict=False
            )
            logging.warning(f"missing keys: {missing_keys}")
            logging.warning(f"unexpected keys: {unexpected_keys}")
        else:
            logging.warning(
                f"model.pt not found in {model_name_or_path}, initialize from scratch"
            )
        return model

    def forward(self, inputs):
        embed = self.clip_model(inputs)

        text_embs = embed["text"]
        image_embs = embed["image"]

        text_fusion = self.act_fn(self.text_fc(text_embs))
        image_fusion = self.act_fn(self.image_fc(image_embs))

        fusion = self.act_fn(
            self.fusion_fc(torch.cat([text_fusion, image_fusion], dim=-1))
        )

        return fusion

def get_collate_fn(processor):
    def collate_fn(batch):
        texts = [item["title"] for item in batch]
        images = [item["image"] for item in batch]
        text_encoded = processor(
            text=texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=77,
        )
        img_encoded = processor(images=images, return_tensors="pt", do_rescale=False)

        return {"text": text_encoded, "image": img_encoded}

    return collate_fn

In [10]:
model_path = model2path["clip"]

# model = CLIPForEmbedding.from_pretrained(model_path)
model = CLIPForFusion.from_pretrained(model_path)
processor = CLIPProcessor.from_pretrained(model_path)


dataset = ShopeeDataset(df, img_dir, split)
loader = DataLoader(
    dataset,
    batch_size=256,
    num_workers=16,
    collate_fn=get_collate_fn(processor),
    shuffle=False,
)

In [11]:
if use_CLIP:
    embs = []
    # get image + text embedding into numpy array
    
    model.eval()
    model.cuda()
    for batch in tqdm(loader):
        with torch.no_grad():
            with torch.amp.autocast("cuda", dtype=torch.float16):
                for k, v in batch.items():
                    batch[k] = {key: val.cuda() for key, val in v.items()}
                emb = model(batch)
                # text_emb, image_emb = emb["text"], emb["image"]
                # emb = torch.cat([text_emb, image_emb], dim=-1)
            embs.append(emb.cpu().detach().numpy())
    
    print(len(embs))
    embs = np.concatenate(embs, axis=0)
    # normalize
    embs /= np.linalg.norm(embs, axis=1, keepdims=True)
    
del model, processor
torch.cuda.empty_cache()


In [12]:
if use_CLIP:
    # for i in range(3):
    #     embs = neighbor_blend(embs, 0.7)
    df["clip"] = retrieval(embs, 0.7, df)

    if CV:
        df["f1_clip"] = df.apply(compute_f1("clip"), axis=1)
        df["recall_clip"] = df.apply(compute_recall("clip"), axis=1)
        df["precision_clip"] = df.apply(compute_precision("clip"), axis=1)
        df["AP_clip"] = df.apply(compute_AP("clip", 50), axis=1)
        
        print(
            df.f1_clip.mean(),
            df.recall_clip.mean(),
            df.precision_clip.mean(),
            df.AP_clip.mean(),
        )

## BGE

In [13]:
from transformers import AutoModel, AutoTokenizer

model_path = model2path["bge"]

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path)

dataset = ShopeeDataset(df, img_dir, split)

def get_collate_fn():
    def collate_fn(batch):
        texts = [item["title"] for item in batch]
        return texts
    return collate_fn

loader = DataLoader(
    dataset=dataset,
    batch_size=256,
    num_workers=16,
    collate_fn=get_collate_fn(),
    shuffle=False,
)

In [14]:
if use_BGE:
    embs = []
    # get text embedding into numpy array
    
    model.eval()
    model.cuda()
    for batch in tqdm(loader):
        with torch.no_grad():
            inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)
            with torch.amp.autocast("cuda", dtype=torch.float16):
                # print(inputs)
                inputs = {k: v.cuda() for k, v in inputs.items()}
                output = model(**inputs)
                emb = output.last_hidden_state[:, 0]
            embs.append(emb.cpu().detach().numpy())
    
    print(len(embs))
    embs = np.concatenate(embs, axis=0)
    # normalize
    embs /= np.linalg.norm(embs, axis=1, keepdims=True)
    
    
del model, tokenizer
torch.cuda.empty_cache()


100%|██████████| 134/134 [05:55<00:00,  2.65s/it]

134


In [24]:
if use_BGE:
    
    df["bge"] = retrieval(embs, 0.7, df)
    if CV:
        df["f1_bge"] = df.apply(compute_f1("bge"), axis=1)
        df["recall_bge"] = df.apply(compute_recall("bge"), axis=1)
        df["precision_bge"] = df.apply(compute_precision("bge"), axis=1)
        df["AP_bge"] = df.apply(compute_AP("bge", 50), axis=1)
        
        print(
            df.f1_bge.mean(),
            df.recall_bge.mean(),
            df.precision_bge.mean(),
            df.AP_bge.mean(),
        )

Chunk Size: 4096, 9 chunks


100%|██████████| 9/9 [00:10<00:00,  1.14s/it]


0.6712451880301725 0.7829629094434313 0.706497872575637 0.683439614045477


## Submission

In [16]:
def combine(cols):
    def combine_(row):
        return np.unique(np.concatenate([row[col] for col in cols]))

    return combine_

cols = ["phash"]
if use_BGE:
    cols.append("bge")
if use_CLIP:
    cols.append("clip")

df["matches"] = df.apply(combine(cols), axis=1)

In [17]:
if CV:
    df["f1"] = df.apply(compute_f1("matches"), axis=1)
    df["recall"] = df.apply(compute_recall("matches"), axis=1)
    df["precision"] = df.apply(compute_precision("matches"), axis=1)
    df["AP"] = df.apply(compute_AP("matches", 50), axis=1)
    
    print(
        df.f1.mean(),
        df.recall.mean(),
        df.precision.mean(),
        df.AP.mean(),
    )

0.7010638791934342 0.790739196451627 0.7430178752254558 0.6737807768892041


In [18]:
submission = df[["posting_id", "matches"]]
submission["matches"] = submission["matches"].apply(lambda x: " ".join(x))
submission.columns = [["posting_id", "matches"]]
submission

/tmp/ipykernel_1233889/3673875693.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission["matches"] = submission["matches"].apply(lambda x: " ".join(x))


,posting_id,matches
0,train_129225211,train_129225211 train_2278313361
1,train_3386243561,train_1816968361 train_1831941588 train_197498...
2,train_2288590299,train_2288590299 train_3803689425
3,train_2406599165,train_1508100548 train_1593362411 train_174495...
4,train_3369186413,train_3369186413 train_921438619
...,...,...
34245,train_4028265689,train_4028265689
34246,train_769054909,train_1006492702 train_1463059254 train_238144...
34247,train_614977732,train_1264798465 train_2325457554 train_269046...
34248,train_3630949769,train_1431563868 train_3419392575 train_363094...


In [19]:
submission.to_csv("submission.csv", index=False)

In [20]:
sub = pd.read_csv("/kaggle/working/submission.csv")
sub

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/submission.csv'